# 0 摘要

过去主流序列传导模型(Sequence Transduction Models)都是基于循环神经网络RNN或者卷积神经网络CNN作为特征提取器，表现比较好的模型则需要在Encoder-Decoder中加入Attention机制。

本文提出一个简单的网络结构，称为**Transformer**，这个结构完全**不需要依赖RNN或者CNN**，而仅仅是建立在**Attention Mechanisms**之上，因此也使得模型更易并行，加快训练速度。

# 1 简介与背景

## 1.1 现有不足

- RNN模型天然的串行序列结构限制计算并行化
- Attention机制往往需要和RNN模型进行组合
- Extended Neural GPU, ByteNet, ConvS2S虽然使用了CNN并行化，但是关联输入与输出之间位置关系的计算操作数会随着距离增大而增大（ConvS2S是线性增加，ByteNet是对数增加），因此不利于模型学习远距离关系

## 1.2 背景

- Transformer将计算位置之间关系的操作数降低为常数级别
- 使用了Multi-Head Attention
- 使用Self-Attention(Intra-Attention)对序列进行表达

# 2 模型结构

## 2.1 Encoder

- 由6层相同的层堆叠
- 每一层包含两个sub-layer:
    - multi-head self-attention menchanism
    - position-wise fully connected feed-forward network
- 在每个sub-layer上使用了residual connection（残差连接），后面接了一个Layer Norm结构($LayerNorm(x+Sublayer(x))$)
- 为了方便这些residual connections，所有的sub-layer包括embedding层的输出维度都是512维

## 2.2 Decoder

- 由6层相同的层堆叠
- 每一层包含三个sub-layer:
    - multi-head self-attention menchanism
    - fully connected feed-forward network
    - masked-multi-head self-attention(保证输出序列中第i位置的元素时，只对前i-1个元素做self-attention)
- 在每个sub-layer上使用了residual connection（残差连接），后面接了一个Layer Norm结构

## 2.3 Attention结构

将给定的一个query和一个key-value pairs集合映射为一个加权输出。

> 类比Seq2Seq中的Attention机制。
> - Decoder端的当前t时刻隐藏层向量$s_t$为query
> - Encoder端的所有timestep下的隐藏层向量构成了key集合$(h_1, h_2, \cdots, h_t)$，value和key在Seq2Seq中是一样的
> - Attention是关于query和key的函数

### 2.3.1 Scaled Dot-Product Attention

输入为一个query和一个集合keys，这两个都是$d_k$维向量，以及values为$d_v$维向量。将query和每个keys进行内积运算，并对结果除以$\sqrt{d_k}$缩放后进入Softmax Layer，得到权重。

![](pictures/scaled-dot-product-attention.png)

在实际计算中，会对多个queries打包为矩阵$Q$后进行并行计算。keys和values也被打包为矩阵$K$和矩阵$V$：

$$\mathcal{Attention}(Q, K, V)=\mathcal{softmax}(\frac{QK^T}{\sqrt{d_k}})V$$

> 常用的Attention计算方法有additive和multiplicative两种方式。前者是将q和k拼接起来训练一层神经网络，后者是计算q和k的内积。这两种的理论复杂度上是同量级的，但实际中dot-product的方式在速度上更快，且space-efficient。

> 作者提到当$d_k$比较小的时候，additive和multiplicative两种attention表现差不多；但当$d_k$变大时，如果都不进行scale，addictive会表现更优。作者猜测是由于$d_k$导致向量内积时出现量纲过大，将内积结果输入softmax后落在了梯度饱和区域，导致没有学好。因此作者才加了$d_k$进行scale。（个人认为有点像做白化或者归一化，或者类似BN的操作，让输入落在非饱和区能够比较好的学习）

### 2.3.2 Multi-Head Attention

作者发现使用多个Attention会使结果更好。因此提出了Multi-Head Attention结构，如下图所示。先将Q、K、V经过不同的线性投影（就是一层线性神经网络不加激活函数）后，再进行Scaled Dot-Product Attention的计算。

![](pictures/multi-head-attention.png)

<img src="https://img-blog.csdn.net/20180806110725729?watermark/2/text/aHR0
cHM6Ly9ibG9nLmNzZG4ubmV0L2xldmlvcGt1/font/5a6L5L2T/fontsize/400/fill/
I0JBQkFCMA==/dissolve/70" width = "400" height = "260" alt="图片名称" 
align=center>